In [1]:
import numpy as np
import tensorflow as tf

In [2]:
N_SAMPLES = 2
NUM_THREADS = 4
k_batchsize = 10  #yes, can be greater than num samples!
                 #gotta figure out for yourself how many times you want to see the same data

# Generating some simple data
# create 1000 random samples, each is a 1D array from the normal distribution (10, 1)
# each vector has 4 dimensions in this example
data = 10 * np.random.randn(N_SAMPLES, 4) + 1
# create 1000 random labels of 0 and 1
target = np.random.randint(0, 2, size=N_SAMPLES)

# Create queue types specified for data and target, shapes specified (4D and scalar)
#XX  queue = tf.FIFOQueue(capacity=50, dtypes=[tf.float32, tf.int32], shapes=[[4], []])
#XX  enqueue_op = queue.enqueue_many([data, target])
#=======================================================
#XX if (k_batchsize > 0) :
#XX     dequeue_op = queue.dequeue_many(k_batchsize) #batch_size at a time, in a list
#XX else :
#XX     dequeue_op = queue.dequeue() #one at a time, not in a list
#=======================================================
# create NUM_THREADS to do enqueue
#XX  qr = tf.train.QueueRunner(queue, [enqueue_op] * NUM_THREADS)

#OR

# This function adds the following to the current Graph:
#    A shuffling queue into which tensors from tensors are enqueued.
#    A dequeue_many operation to create batches from the queue.
#    A QueueRunner to QUEUE_RUNNER collection, to enqueue the tensors from tensors.
imageBatch, labelBatch = tf.train.shuffle_batch(
    [data, target], batch_size=k_batchsize,
    num_threads=NUM_THREADS,
    enqueue_many=True, #IMPORTANT to get right, default=False
    capacity=10,  #1000,
    min_after_dequeue=5) #500



In [3]:
with tf.Session() as sess:
    # Create a coordinator, launch the queue runner threads.
    coord = tf.train.Coordinator()
    #XX  enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    enqueue_threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    for step in xrange(1): 
        if coord.should_stop():
            break
        #XX  data_batch, label_batch = sess.run(dequeue_op)
        data_batch, label_batch = sess.run([imageBatch, labelBatch])
        
        print('------  step: ' + str(step))
        print('data_batch tf.shape is ' + str(tf.shape(data_batch))) # 2, its a 2D array
        print('batch size is ' + str(data_batch.shape[0])) # (rows, colums)
        print('data size is ' + str(data_batch.shape[1])) #the length of a column
        print label_batch
        print data_batch

        
        
    coord.request_stop()
    coord.join(enqueue_threads)

------  step: 0
data_batch tf.shape is Tensor("Shape:0", shape=(2,), dtype=int32)
batch size is 10
data size is 4
[1 1 1 1 1 1 1 1 1 1]
[[-20.01829143   4.59999121  14.93254106   7.8250762 ]
 [-20.01829143   4.59999121  14.93254106   7.8250762 ]
 [ -5.77860222  -3.59003689   4.01297669   0.07831853]
 [-20.01829143   4.59999121  14.93254106   7.8250762 ]
 [-20.01829143   4.59999121  14.93254106   7.8250762 ]
 [-20.01829143   4.59999121  14.93254106   7.8250762 ]
 [ -5.77860222  -3.59003689   4.01297669   0.07831853]
 [-20.01829143   4.59999121  14.93254106   7.8250762 ]
 [ -5.77860222  -3.59003689   4.01297669   0.07831853]
 [ -5.77860222  -3.59003689   4.01297669   0.07831853]]
